In [1]:
import pandas as pd
import numpy as np 
from text_analytics import * 
import matplotlib.pyplot as plt 


Bad key "text.kerning_factor" on line 4 in
C:\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [7]:
data = pd.read_csv('../Data/data_with_sentiment.csv')
data.head()

,SupportTicketID,CustomerID,DateCreated,DateCompleted,Escalated,Theme,Theme_Desc,Text,Sentiment,Process Time
0,16582,10007217,2018-06-30,2018-04-07,1,price,cost of the goods to customers. compare to co...,A screwdriver costs over a hundred dollars? Wh...,negative,-84
1,13830,10005984,2018-07-15,2018-07-18,0,speed,consider the delivery speed.,Anyway that I can rush overnight ship my order?,neutral,3
2,53794,10022738,2018-10-14,2018-10-18,0,features,"product features, website features, checkout f...",Are the shelves removable from the wine fridge...,neutral,4
3,95013,10040142,2018-01-11,2018-10-11,0,price,cost of the goods to customers. compare to co...,Are there any Christmas sales coming up?,neutral,273
4,32660,10013924,2018-06-06,2018-06-06,1,design,"product design, website design",Are there any other colors in the bathroom sin...,neutral,0


In [25]:
type(data['Process Time'][0])

numpy.int64

Data Cleaning Notes: 
    - Remove the NAs in the text column 
    - Clean the negative Process Time values by replacing with the mean of positive times in the same escalated and theme 

In [62]:
cleaned = data.dropna()

In [63]:
def impute_process_time(TicketID):
    temp = data[data['SupportTicketID'] == TicketID]
    escalated = temp['Escalated'].iloc[0]
    theme = temp['Theme'].iloc[0]
    positive_grouped = data[(data['Process Time'] >= 0) & (data['Theme'] == theme) & (data["Escalated"] == escalated)]
    average = np.mean(positive_grouped['Process Time'])
    return average

In [67]:
cleaned_processTimes = list()
for row in cleaned.iterrows():
    if row[1]['Process Time'] < 0:
        idVal = row[1]['SupportTicketID']
        val = impute_process_time(idVal)
        cleaned_processTimes.append(val)
    else:
        cleaned_processTimes.append(row[1]['Process Time'])

In [69]:
final = cleaned.copy()
final['Process Time'] = cleaned_processTimes
final.head()

,SupportTicketID,CustomerID,DateCreated,DateCompleted,Escalated,Theme,Theme_Desc,Text,Sentiment,Process Time
0,16582,10007217,2018-06-30,2018-04-07,1,price,cost of the goods to customers. compare to co...,A screwdriver costs over a hundred dollars? Wh...,negative,2.0
1,13830,10005984,2018-07-15,2018-07-18,0,speed,consider the delivery speed.,Anyway that I can rush overnight ship my order?,neutral,3.0
2,53794,10022738,2018-10-14,2018-10-18,0,features,"product features, website features, checkout f...",Are the shelves removable from the wine fridge...,neutral,4.0
3,95013,10040142,2018-01-11,2018-10-11,0,price,cost of the goods to customers. compare to co...,Are there any Christmas sales coming up?,neutral,273.0
4,32660,10013924,2018-06-06,2018-06-06,1,design,"product design, website design",Are there any other colors in the bathroom sin...,neutral,0.0


In [70]:
final.to_csv('cleaned.csv', index = False)